<img src="logounav.png" width="150" img style="float: right;"> 

**Data collection from Instagram.**<br>
Author: Lucía Colín Cosano

First, the necessary libraries are imported.

In [7]:
import json
from lxml import html
import re
import csv
import numpy as np

from selenium import webdriver
import time
import os
import time
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (NoSuchElementException,TimeoutException,WebDriverException,)

Configuring Chrome Driver.

In [8]:
WAIT_TIME = 10

# Chrome driver should be in
executable_path=os.path.join('C:/Program Files/Google/Chrome/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-notifications')

preferences = {
    "profile.default_content_setting_values.notifications" : 1,
    "profile.default_content_setting_values.location": 1,
    "profile.managed_default_content_settings.images": 1,
    }
options.add_experimental_option("prefs", preferences)

chrome_options = Options()
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")

driver = webdriver.Chrome(executable_path = executable_path,chrome_options = chrome_options,)

C:\Users\lulac\AppData\Local\Temp\ipykernel_1580\1192129472.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = executable_path,chrome_options = chrome_options,)
C:\Users\lulac\AppData\Local\Temp\ipykernel_1580\1192129472.py:22: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = executable_path,chrome_options = chrome_options,)


Sometimes, the error shown in the image below appears, which can be fixed by adding the following instruction:
    
driver = webdriver.Chrome(ChromeDriverManager().install())

<img src="error1.png">

The screen is maximized for better work. The number 20 defines a waiting time.

In [9]:
driver.maximize_window()
driver.implicitly_wait(20) 

Once the browser is opened, the URL from which data is to be obtained is defined. In this case, the goal is to extract data from an Instagram hashtag.

In [10]:
driver.wait = WebDriverWait(driver, WAIT_TIME)
url = 'https://www.instagram.com/?hl=es'
driver.get(url)

Once you enter Instagram, it is necessary to accept the cookies.

In [11]:
cookies_button = driver.find_element(By.CSS_SELECTOR,"button[class='_a9-- _a9_1']")
cookies_button.click()
time.sleep(1)

Both the username and password are defined. Some input statements are defined that could be used to ask the user to enter their credentials. In this case, they are directly written to test the functionality.

The button where you need to click to enter the credentials is found, and the login process is initiated.

In [12]:
USERNAME = '***' # Insert your user name and password
PASSWORD = '***'

usr = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.NAME, "username")))
usr.send_keys(USERNAME)

password = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.NAME, "password")))
password.send_keys(PASSWORD)
time.sleep(4)

The following error is due to the definition of the functions provided by Selenium for element selection. They used to follow the format: driver.find_elements_by_css_selector, driver.find_elements_by_xpath. This library has evolved, generalizing this function to driver.find_elements() and the first argument of the function determines what type of search you want to perform.

It is necessary to import from selenium.webdriver.common.by import By. Two examples of implementation are:

- driver.find_element(By.CSS_SELECTOR, "button[class='_a9-- _a9_1']")
- driver.find_element(By.XPATH, '//button[text()="._9AH"]')

<img src="error2.png">

In [13]:
driver.find_elements(By.CSS_SELECTOR,('button'))[1].click()
time.sleep(2)

Once logged in, a dialog box appears asking if you want to save the information, and the "Not Now" button is clicked. The same is done for enabling notifications. 

In [14]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='_acan _acap _acas _aj1-']")))
guardar_info_button=driver.find_element(By.CSS_SELECTOR,"button[class='_acan _acap _acas _aj1-']")
guardar_info_button.click()
time.sleep(2)

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[class='_a9-- _a9_1']")))
notificaciones_button=driver.find_element(By.CSS_SELECTOR,"button[class='_a9-- _a9_1']")
notificaciones_button.click()
time.sleep(5)

The desired hashtag to scrape is defined, and the Instagram search bar is located.

In [15]:
HASHTAG='#mundialqatar2022'

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"a[href='#']")))
busqueda_button=driver.find_element(By.CSS_SELECTOR,"a[href='#']")
busqueda_button.click()

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[aria-label='Buscar entrada']")))
intro_hashtag=driver.find_element(By.CSS_SELECTOR,"input[aria-label='Buscar entrada']")
intro_hashtag.clear()
intro_hashtag.send_keys(HASHTAG)
time.sleep(5)
intro_hashtag.send_keys(Keys.ENTER)
time.sleep(5)
intro_hashtag.send_keys(Keys.ENTER)
time.sleep(5)

Once we are on the posts corresponding to the selected hashtag, we want to obtain the link of the post, the user who posted it, the location (which is often not available), the time since the post was made, the number of likes, and finally, the caption.

In [16]:
enlace_post = driver.find_elements(By.TAG_NAME,'a')
post = [i.get_attribute('href') for i in enlace_post]
print(len(post))

url_post=[]
for i in post:
    if '/p/' in i:
        url_post.append(i)  

45


The URLs of the obtained posts are iterated, and in the following loop, each post is accessed to retrieve the corresponding information. Additionally, the number of posts from which you want to obtain information is defined. It is recommended that this number not be very high as it would result in long waiting times.

In [17]:
users=[] #usuarios de los post
times=[] #fecha en la que se publicó el post
locations=[] #localizaciones de los post
likes=[] #numero de likes
pie=[] #pie de foto

num_publi_deseadas=4

#for i in url_post:
for i in url_post[0:num_publi_deseadas]:

    driver.get(i)
    
    #Para obtener los usuarios en una lista
    try:
        users.append(driver.find_element(By.CSS_SELECTOR, "a[class='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _acan _acao _acat _acaw _aj1- _a6hd']").text)
    except NoSuchElementException:
        users.append("Publicación no disponible")
        
    #Para obtener las localizaciones en una lista    
    try:
        locations.append(driver.find_element(By.CSS_SELECTOR, "a[class='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _aaqk _a6hd']").text)
    except:
        locations.append(" ")
        
    #Para obtener el tiempo transcurrido desde que se publicó en una lista    
    try:
        times.append(driver.find_element(By.CSS_SELECTOR,"time[class='_aaqe']").text)     
    except:
        times.append("No available")  
    
    #Para obtener el número de likes que hay en cada publicación en una lista
    try:
        likes.append(driver.find_element(By.CSS_SELECTOR,"div[class='_aacl _aaco _aacw _aacx _aada _aade']").text)
    except:
        likes.append(' ')
        
    #Para obtener el pie de foto en una lista
    try:
        pie.append(driver.find_element(By.CSS_SELECTOR,"div[class='_a9zr']").text)
    except:
        pie.append(' ')
    time.sleep(3)

Finally, the DataFrame is created and printed.

In [18]:
df = pd.DataFrame(list(zip(url_post,users,locations, times, likes,pie)), columns=['Link al Post','Nombre del Usuario', 'Localización', 'Tiempo desde la publiación', 'Número de likes/reproducciones','Pie de foto'])
df

,Link al Post,Nombre del Usuario,Localización,Tiempo desde la publiación,Número de likes/reproducciones,Pie de foto
0,https://www.instagram.com/p/CmUoG38ADob/,leomessiandfamily,,HACE 21 HORAS,44.895 Me gusta,leomessiandfamily\nThis is Love 💙🥺\n\n#messi #...
1,https://www.instagram.com/p/CmWczJpOTmt/,daleseleccion,Lusail Stadium,HACE 4 HORAS,2091 Me gusta,daleseleccion\nNuestro Ángel de la guarda ❤️🇦🇷...
2,https://www.instagram.com/p/CmUe9GJukZl/,daleseleccion,Lusail Stadium,HACE 23 HORAS,14.996 Me gusta,daleseleccion\nEl sueño se hizo realidad\n\n#m...
3,https://www.instagram.com/p/CmWT_WrucV-/,depoweb,,HACE 6 HORAS,1521 Me gusta,"depoweb\n🤌 ANDA PA' ALLÁ, KYLIAN\n\n👋Luego de ..."


## COMMON ERRORS

Often, it was not possible to click buttons like the one for accepting notifications. This is due to waiting times. It was necessary to adjust these values until it never failed.

Another error that has continuously occurred is InvalidSelectorException, as shown below. This is because sometimes the specified XPath is not valid. Due to the continuous failures, it was decided to exclusively use the CSS selector.
<img src="error4.png">

Another aspect to highlight is the importance of choosing between find_element and find_elements. The difference between them is as follows. It is important to keep this in mind as it can prevent the error shown in the image below.

- findElement: Command used to uniquely identify a web element within the web page.
- findElements: Command used to identify a list of web elements within the web page.

<img src="error6.png">